# Image_extraction_for_detection

In [ ]:
!pip uninstall tensorflow --y
!pip install tensorflow==1.12.0
!pip install pytesseract
!pip install tensorflow-object-detection-api

import tensorflow as tf
import numpy as np # linear algebra and array

import pandas as pd # data processing, JSON file I/O
import urllib
import requests # data from url 
import json
import random

from PIL import Image
import matplotlib.pyplot as plt # plotting the image 
import cv2

import pickle # Save the data locally
import io 

  Using cached https://files.pythonhosted.org/packages/22/cc/ca70b78087015d21c5f3f93694107f34ebccb3be9624385a911d4b52ecef/tensorflow-1.12.0-cp36-cp36m-manylinux1_x86_64.whl


In [ ]:
# Read the data 
data = pd.read_json('Indian_Number_plates.json', lines=True)
pd.set_option('display.max_colwidth', None)

# Data Preprocessing
# Delete the empty column
del data['extras']

# Extract the Box of number plate via points provided
data['points'] = data.apply(lambda row: row['annotation'][0]['points'], axis=1)

# Data cleaning
# And drop the rest of the annotation info
del data['annotation']

In [ ]:
Images = []
Plates = []

def downloadTraining(df):

    for index, row in df.iterrows():

        # Get the image from the URL
        # URL is the first "content" in a line
        resp = urllib.request.urlopen(row[0]) 
        im = np.array(Image.open(resp))

        # We append the image to the training input array
        Images.append(im)  

        # Points of Number plate Boundary 
        # Multiplying given points with height and width to get the actual co-ordinates
        x_point_top = row[1][0]['x']*im.shape[1]
        y_point_top = row[1][0]['y']*im.shape[0]
        x_point_bot = row[1][1]['x']*im.shape[1]
        y_point_bot = row[1][1]['y']*im.shape[0]

        # Cut the plate from the image and keep it seperately
        completeImage = Image.fromarray(im)
        plateImage = completeImage.crop((x_point_top, y_point_top, x_point_bot, y_point_bot))
        Plates.append(np.array(plateImage))

downloadTraining(data)

In [ ]:
# Using pickle to store the python list locally so that extracting images from url is not required again
# Python lists of car and number plate saved in list
print("Data Loading...Wait")
with open('Images_array_train', 'wb') as f:
    pickle.dump(Images, f)
with open('Plates_array_train', 'wb') as f:
    pickle.dump(Plates, f)  

print("Data Saved")

In [ ]:
# The Locally saved python list can be accesed easily for images
with open('Images_array_train', 'rb') as f:
    mylist1 = pickle.load(f)

with open('Plates_array_train', 'rb') as f:
    mylist2 = pickle.load(f)

# Create figure and axes
fig, ax = plt.subplots(2, 1, constrained_layout=True)

# Set title
ax[0].set_title('Input Image')
ax[1].set_title('Output Image')

# Display the images
# Car
tt=random.randint(0, 235)
ax[0].imshow(mylist1[tt])
# Number plate
ax[1].imshow(mylist2[tt])

plt.show()

In [ ]:


import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import random

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import pytesseract

from tensorflow.core.protobuf import saved_model_pb2

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

## Env setup

In [ ]:
# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

# This is needed to display the images.
%matplotlib inline

from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util
from tensorflow.python.util import compat


In [ ]:
def yo_make_the_conversion(img_data, count):
    img = img_data
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.threshold(gray, 0, 255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    gray = cv2.medianBlur(gray, 3)
    path_png='png_tesseract'
    count += 1
    filename = os.path.join(path_png,'image{}.png'.format(count))
    cv2.imwrite(filename, gray)
    return filename

In [ ]:
def check_array(tex):
    side=tex
    az='ABCDEFGHIJKLMNOPQRSTUVWXYZ1234567890'
    i=0
    yo=len(az)
    txt=''
    for i in range(0,yo):
        if side==az[i]:
            txt=az[i]
            break
    return txt   

In [ ]:
def catch_rectify_plate_characters(text):
    tex = text
    out1=[]
    size=len(tex)
    for i in range(0,size):
      if tex[i]==check_array(tex[i]):
        out1.append(tex[i])
    yup=''.join(str(e) for e in out1)    
    return yup

In [ ]:
MODEL_NAME = ''
PATH_TO_CKPT = MODEL_NAME + 'frozen_inference_graph.pb'
PATH_TO_LABELS = os.path.join('training', 'object-detection.pbtxt')
NUM_CLASSES = 1

### Load a (frozen) Tensorflow model into memory.

In [ ]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  with tf.compat.v1.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = compat.as_bytes(fid.read())
    sm = saved_model_pb2.SavedModel()
    sm.ParseFromString(serialized_graph)
    tf.import_graph_def(sm.meta_graphs[0].graph_def)

In [ ]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

## Detection

In [ ]:
PATH_TO_TEST_IMAGES_DIR = 'png_tesseract/test_ram'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.JPEG'.format(i)) for i in range(0,15) ]
IMAGE_SIZE = (12, 8)
TEST_DHARUN=os.path.join('number_plate')
count = 0

In [ ]:
with detection_graph.as_default():
  with tf.Session(graph=detection_graph) as sess:
    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
    detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
    detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
    detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
    num_detections = detection_graph.get_tensor_by_name('num_detections:0')

    for image_path in TEST_IMAGE_PATHS:
      image = Image.open(image_path) 
      image_np = cv2.imread(image_path,1)
      image_np_expanded = np.expand_dims(image_np, axis=0)
      (boxes, scores, classes, num) = sess.run(
          [detection_boxes, detection_scores, detection_classes, num_detections],
          feed_dict={image_tensor: image_np_expanded})
      ymin = boxes[0,0,0]
      xmin = boxes[0,0,1]
      ymax = boxes[0,0,2]
      xmax = boxes[0,0,3]
      (im_width, im_height) = image.size
      (xminn, xmaxx, yminn, ymaxx) = (xmin * im_width, xmax * im_width, ymin * im_height, ymax * im_height)
      cropped_image = tf.image.crop_to_bounding_box(image_np, int(yminn), int(xminn),int(ymaxx - yminn), int(xmaxx - xminn))
      img_data = sess.run(cropped_image)
      count = 0
      
      config = ('-l eng --oem 1 --psm 3')
      filename = yo_make_the_conversion(img_data, count)

      pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR'
      pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
      text = pytesseract.image_to_string(img_data,config=config)
      print('CHARCTER RECOGNITION : ',catch_rectify_plate_characters(text))
      vis_util.visualize_boxes_and_labels_on_image_array(
          image_np,
          np.squeeze(boxes),
          np.squeeze(classes).astype(np.int32),
          np.squeeze(scores),
          category_index,
          use_normalized_coordinates=True,
          line_thickness=5)
      
      
      plt.figure(figsize=IMAGE_SIZE)
      plt.subplot(1,2,1)
      plt.imshow(image)
      plt.subplot(1,2,2)
      plt.imshow(img_data)
      plt.show()